## Cppyy + Numba? 

In [ ]:
import numba
import cppyy
import cppyy.numba_ext

An experimental feature that combines the best of both worlds

In [ ]:
cppyy.cppdef("""
#include <vector>
struct Atom {
float x;
float y;
float z;
};
std::vector<Atom> atoms = {{1, 2, 3}, {2, 3, 4}, {3, 4, 5}, {4, 5, 6}, {5, 6, 7}};
""")

In [ ]:
@numba.njit
def lj_numba_scalar(r):
    sr6 = (1./r)**6
    pot = 4.*(sr6*sr6 - sr6)
    return pot

JIT compile a Python function with Cppyy C++ objects in them

In [ ]:
@numba.njit
def distance_numba_scalar(atom1, atom2): # The arguments here are structs
    dx = atom2.x - atom1.x
    dy = atom2.y - atom1.y
    dz = atom2.z - atom1.z
    r = (dx * dx + dy * dy + dz * dz) ** 0.5
    return r

In [ ]:
@numba.njit
def distance_numba_python(atom1, atom2): # The arguments here are structs
    dx = atom2[0] - atom1[0]
    dy = atom2[1] - atom1[1]
    dz = atom2[2] - atom1[2]
    r = (dx * dx + dy * dy + dz * dz) ** 0.5
    return r

In [ ]:
def potential_numba_calculator(cluster):
    energy = 0.0
    for i in range(cluster.size() - 1):
        for j in range(i + 1, cluster.size()):
            r = distance_numba_cppyy(cluster[i], cluster[j])
            e = lj_numba_scalar(r)
            energy += e
    return energy

In [ ]:
def potential_calculator(cluster):
    energy = 0.0
    for i in range(len(cluster) - 1):
        for j in range(i + 1, len(cluster)):
            r = distance_python(cluster[i], cluster[j])
            e = lj_scalar(r)
            energy += e
    return energy

In [ ]:
potential_numba_calculator(cppyy.gbl.atoms)